In [1]:
import numpy
import math
import matplotlib.pyplot as pyplot

In [2]:
def stat_chars(S, text):
    median = numpy.median(S)
    variance = numpy.var(S)
    devation = math.sqrt(variance)
    print(text)
    print('Математичне очікування =', median)
    print('Дисперсія =', variance)
    print('Середньоквадратичне відхилення =', devation)
    print('')

In [3]:
def graphics(models, hist=False):
    for key, value in models:
        if hist:
            pyplot.hist(value, label=key, alpha=0.5)
            continue
        pyplot.plot(value, label=key, alpha=0.9)
    pyplot.legend()
    pyplot.show()

In [4]:
def lin_model(n):
    sample = numpy.zeros(n)
    for i in range(n):
        sample[i] = 0.001*i
    return sample

In [5]:
def rand_exp_model(n, dsig):
    sample = numpy.random.exponential(dsig, n)
    return sample

In [6]:
def additive_model(sample1, sample2, n):
    sample = numpy.zeros(n)
    for i in range(n):
        sample[i] = sample1[i] + sample2[i]
    return sample

In [7]:
def add_anomalies(sample1, sample2, n, an, dsig):
    sample = numpy.zeros(n)
    anomalies = numpy.random.exponential((3*dsig), an)
    
    for i in range(n):
        sample[i] = sample1[i]
        
    for i in range(an):
        num = numpy.random.randint(1, n)
        sample[num] = sample2[num] + anomalies[i]
        
    return sample

In [8]:
def speed_MNK(S):
    iter = len(S)
    Yin = numpy.zeros((iter, 1))
    F = numpy.ones((iter, 3))
    for i in range(iter):
        Yin[i, 0] = float(S[i])
        F[i, 1] = float(i)
        F[i, 2] = float(i * i)
    FT = F.T
    FFT = FT.dot(F)
    FFTI = numpy.linalg.inv(FFT)
    FFTIFT = FFTI.dot(FT)
    C = FFTIFT.dot(Yin)
    Yout = F.dot(C)
    return C[1,0]

In [9]:
def remove_anomalies_MNK(S, Q, n_Wind):
    S0 = numpy.zeros(len(S))
    for i in range(len(S)):
        S0[i] = S[i]
    
    iter = len(S0)
    j_Wind = math.ceil(iter-n_Wind)+1
    S0_Wind = numpy.zeros((n_Wind))

    Speed_standart = speed_MNK(S0)
    Yout_S0 = anti_aliasing_MNK(S0)

    for j in range(j_Wind):
        for i in range(n_Wind):
            l = (j+i)
            S0_Wind[i] = S0[l]

        Speed = speed_MNK(S0_Wind)
        dS = numpy.var(S0_Wind)
        scvS = math.sqrt(dS)

        Speed_standart_1 = abs(Speed_standart * math.sqrt(iter))
        Speed_1 = abs(Q * Speed_standart * math.sqrt(n_Wind) * scvS)

        if Speed_1 > Speed_standart_1:
            S0[l] = Yout_S0[l,0]
            
    return S0

In [ ]:
def anti_aliasing_MNK(S):
    iter = len(S)
    Yin = numpy.zeros((iter, 1))
    F = numpy.ones((iter, 3))
    for i in range(iter):
        Yin[i, 0] = float(S[i])
        F[i, 1] = float(i)
        F[i, 2] = float(i * i)
    FT = F.T
    FFT = FT.dot(F)
    FFTI = numpy.linalg.inv(FFT)
    FFTIFT = FFTI.dot(FT)
    C = FFTIFT.dot(Yin)
    Yout = F.dot(C)
    return Yout

In [11]:
def anti_aliasing_recurrent(S):
    iter = len(S)
    Yin = numpy.zeros((iter, 1))
    YoutAB = numpy.zeros((iter, 1))
    T = 1
    for i in range(iter):
        Yin[i, 0] = float(S[i])
    
    Yspeed_retro = (Yin[1, 0]-Yin[0, 0])/T
    Yextra = Yin[0, 0] + Yspeed_retro
    alfa = 2*(2*1-1)/(1*(1+1))
    beta = (6/1)*(1+1)
    YoutAB[0, 0] = Yin[0, 0] + alfa*(Yin[0, 0])
    
    for i in range(1, iter):
        YoutAB[i,0] = Yextra + alfa*(Yin[i, 0]- Yextra)
        Yspeed = Yspeed_retro + (beta/T)*(Yin[i, 0]- Yextra)
        Yspeed_retro = Yspeed
        Yextra = YoutAB[i,0] + Yspeed_retro
        alfa = (2 * (2 * i - 1)) / (i * (i + 1))
        beta = 6 /(i* (i + 1))
        
    return YoutAB